In [18]:
import sys
import os
from nltk.corpus import stopwords
from math import sqrt
import numpy as np
import tkinter as tk
from tkinter import *
from tkinter import ttk,messagebox
from tkinter.scrolledtext import *
import tkinter.filedialog
from tkinter.filedialog import askopenfile


In [19]:
class matrix (object):
    
    def normalize_matrix (self,M):
        col_sums = np.sum (M, axis = 0)
        if col_sums.all != 0:      
            M_normal = M / col_sums
        return M_normal
    
    def weight_matrix (self,n):
        X = np.zeros ((n, 1)) + 1 / n
        return X
    
    def calculate_rank (self, M, d, n, prints = False):
        N = M.shape[0]
        X = self.weight_matrix (N)
        
        for i in range (n):
            X = (1 - d) / N + d * np.dot (M, X)
            if prints == True:
                print (i)
                print (X)
        return X

    #for matching rankes in order they appear in text
    def list_ (self, M):
        dict = {}
        for i in range (len (M)):
            dict.update({i: None})
        for i in range (len (M)):
            x = M[i] + 1
            for j in range (len (M)):
                if dict[j] == None:
                    x = x - 1
                if x == 0:
                    dict[j] = i
                    break
        return dict
    
    def ordered (self, M, d, n, _print = False, prints = False):
        N = M.shape[0]
        intermediate_order = []
        for i in range (N):
            M_order = self.normalize_matrix (M)
            X = self.calculate_rank (M_order, d, n, prints)
            if _print == True:
                print (X)
            index = 0
            for cntr in range (N - i):
                if (X[cntr][0] > X[index][0]):
                    index = cntr
            intermediate_order.append (index)
            M = np.delete (M, index, 0)
            M = np.delete (M, index, 1)
        ordered = self.list_ (intermediate_order)
        return ordered    

In [20]:
class summarizer (matrix):
    
    def __init__ (self, text_file):
        self.text_file = text_file
    
    def extract_sentences(self):
        file = open(self.text_file, "r",encoding = 'utf-8')
        file_content = file.readlines()
        text = file_content[0].split(". ")
        sentences = []

        for sentence in text:
            sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
        file.close()
        lines_in_summary = int(.4 * len(sentences))
        return sentences, lines_in_summary

    def cosine_similarity(self, sent_vec1, sent_vec2):
        return (np.dot(sent_vec1, sent_vec2) / (sqrt(np.dot(sent_vec1, sent_vec1)) * sqrt(np.dot(sent_vec2, sent_vec2))))

    def sentence_similarity(self, sent1, sent2, stopwords=None):
        if stopwords is None:            
            stopwords = []

        sent1 = [word.lower() for word in sent1]
        sent2 = [word.lower() for word in sent2]

        word_list = list(set(sent1 + sent2))
        vec_size = len(word_list)

        sent_vec1 = [0] * vec_size
        sent_vec2 = [0] * vec_size
        
        for word in sent1:
            if word in stopwords:   
                continue             
            sent_vec1[word_list.index(word)] += 1

        for word in sent2:
            if word in stopwords:
                continue
            sent_vec2[word_list.index(word)] += 1
            
        return self.cosine_similarity(sent_vec1, sent_vec2)
        
    def build_similarity_matrix(self, sentences, stop_words):
        similarity_matrix = np.zeros((len(sentences), len(sentences)))

        for i in range(len(sentences)):
            for j in range(len(sentences)):
                if i == j: 
                    continue 
                similarity_matrix[i][j] = self.sentence_similarity(sentences[i], sentences[j], stop_words)
        return similarity_matrix

    def generate_summary(self):   
        stop_words = stopwords.words('english')
        summarize_text = []
        
        sentences, lines_in_summary =  self.extract_sentences()
        sentence_similarity_matrix = self.build_similarity_matrix(sentences, stop_words)
        ranks = matrix.ordered (self, sentence_similarity_matrix, 0.85, 20)

        i = j = 0
        while True:
            if (ranks[i] < lines_in_summary):
                summarize_text.append (" ".join(sentences[i]))
                j = j + 1
            if j == lines_in_summary:
                break
            i = i + 1
        print(ranks)
        print("Summarize Text: \n", ". ".join(summarize_text) + '.')
        summary = ". ".join(summarize_text) + '.'
        return summary
    

In [21]:

 # Structure and Layout
window = Tk()
window.title("Text Summarizer")
window.geometry("700x400")
window.config(background='#000000')
window.state("zoomed")
window.resizable(0,0)

style = ttk.Style(window)
style.configure('lefttab.TNotebook', tabposition='wn',)

# TAB LAYOUT
tab_control = ttk.Notebook(window,style='lefttab.TNotebook')
tab1 = ttk.Frame(tab_control)
tab2 = ttk.Frame(tab_control)

# ADD TABS TO NOTEBOOK
tab_control.add(tab1, text=f'{"From User Input":^25s}')
tab_control.add(tab2, text=f'{"From text file":^30s}')


label1 = Label(tab1, text= 'Summarize input text',padx=5, pady=5)
label1.grid(column=0, row=0)

label2 = Label(tab2, text= 'Summarize text from file',padx=5, pady=5)
label2.grid(column=0, row=0)

tab_control.pack(expand=1, fill='both')

# Heading
l1=Label(tab1,text="Enter Text To Summarize")
l1.grid(row=1,column=0)

entry=Text(tab1,height=17, width = 155, wrap = "word")
entry.grid(row=2,column=0,columnspan=2,padx=5,pady=5)

# Main Summary
#create a file save data, pass to generate summary destroy file
def get_summary():
    raw_text = str(entry.get('1.0',tk.END))
    file = open('text.txt', "w", encoding = 'utf-8')
    for line in raw_text:
        file.write(line)
    file.close()
    obj = summarizer("text.txt")
    final_text = obj.generate_summary()
    print(final_text)
    os.remove("text.txt")
    result = '\nSUMMARY\n{}'.format(final_text)
    tab1_display.insert(tk.END,result)

#  Widgets
def clear_text():
    entry.delete('1.0',END)

def clear_display_result():
    tab1_display.delete('1.0',END)

# Buttons
button1=Button(tab1,text="Reset",command=clear_text, width=12,bg='#03A9F4',fg='#fff')
button1.grid(row=4,column=0,padx=10,pady=10)

button2=Button(tab1,text="Clear Result", command=clear_display_result,width=12,bg='#03A9F4',fg='#fff')
button2.place(relx = 0.5, rely = 0.49, anchor = CENTER)

button3=Button(tab1,text="Summarize",command=get_summary, width=12,bg='#03A9F4',fg='#fff')
button3.grid(row=4,column=1,padx=10,pady=10)

# Display
tab1_display = Text(tab1, height = 18, width = 155, wrap = "word")
tab1_display.grid(row=7,column=0, columnspan=2,padx=5,pady=5)

# Heading
l1=Label(tab2,text="Browse text file")
l1.grid(row=10,column=2)

final_text = str()

def open_file():
    file = askopenfile(mode ='r', filetypes =[('Text Files', '*.txt')]) 
    if file is None:
        messagebox.showerror("Error", "Error opening file")
    else: 
        obj = summarizer(file.name)
        global final_text
        final_text = obj.generate_summary() 
    return file

def print_summary():
    global final_text
    result = '\nSummary:{}'.format(final_text)
    tab2_display_text.insert(tk.END,result)

def clear_display_result_():
    tab2_display_text.delete('1.0',END)

button1 = Button(tab2, text ='Open', command = open_file,  width=12,bg='#03A9F4',fg='#fff') 
button1.grid(row=11,column=2,padx=10,pady=10)
    
button2=Button(tab2,text="Summarize",command=print_summary, width=12,bg='#03A9F4',fg='#fff')
button2.grid(row = 12,column=2,padx=10,pady=10)

button3=Button(tab2,text="Clear Result", command=clear_display_result_,width=12,bg='#03A9F4',fg='#fff')
button3.grid(row = 11,column=0,padx=10,pady=10)

tab2_display_text = ScrolledText(tab2,height=17, width = 153, wrap = "word")
tab2_display_text.grid(row=9,column=0, columnspan=3,padx=5,pady=5)

window.mainloop()


